In [1]:
import torch

In [38]:
def mask_finished_scores(score: torch.Tensor,
                         flag: torch.Tensor) -> torch.Tensor:
    """
    If a sequence is finished, we only allow one alive branch. This function
    aims to give one branch a zero score and the rest -inf score.
    Args:
        score (torch.Tensor): A real value array with shape
            (batch_size * beam_size, beam_size).
        flag (torch.Tensor): A bool array with shape
            (batch_size * beam_size, 1).
    Returns:
        torch.Tensor: (batch_size * beam_size, beam_size).
    """
    beam_size = score.size(-1)
    zero_mask = torch.zeros_like(flag, dtype=torch.bool)
    if beam_size > 1:
        unfinished = torch.cat((zero_mask, flag.repeat([1, beam_size - 1])),
                               dim=1)
        finished = torch.cat((flag, zero_mask.repeat([1, beam_size - 1])),
                             dim=1)
    else:
        unfinished = zero_mask
        finished = flag
    print(unfinished)
    print(finished)
    score.masked_fill_(unfinished, -float('inf'))
    score.masked_fill_(finished, 0)
    return score

In [58]:
score = torch.randn((2, 3))
flag = torch.ones((2, 1), dtype=torch.bool)
flag[1] = False
print(flag)
print(score)
print(flag.repeat([1, 2]))

tensor([[ True],
        [False]])
tensor([[-0.8841,  0.7381, -0.9986],
        [ 0.2675, -0.7971,  0.3798]])
tensor([[ True,  True],
        [False, False]])


In [59]:
r  = mask_finished_scores(score, flag)
print(r)
print(score)

tensor([[False,  True,  True],
        [False, False, False]])
tensor([[ True, False, False],
        [False, False, False]])
tensor([[ 0.0000,    -inf,    -inf],
        [ 0.2675, -0.7971,  0.3798]])
tensor([[ 0.0000,    -inf,    -inf],
        [ 0.2675, -0.7971,  0.3798]])


In [55]:
import paddle

score = paddle.randn((2, 3))
flag = paddle.ones((2, 1), dtype='bool')
flag[1] = False
print(flag)
print(score)
print(flag.tile([1, 2]))

Tensor(shape=[2, 1], dtype=bool, place=CUDAPlace(0), stop_gradient=True,
       [[True ],
        [False]])
Tensor(shape=[2, 3], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [[ 2.05994511,  1.87704289,  0.01988174],
        [-0.40165186,  0.77547729, -0.64469045]])
Tensor(shape=[2, 2], dtype=bool, place=CUDAPlace(0), stop_gradient=True,
       [[True , True ],
        [False, False]])


In [56]:
paddle.bool = 'bool'

def masked_fill(xs:paddle.Tensor, mask:paddle.Tensor, value:float):
    print(xs)
    trues = paddle.ones_like(xs) * value
    assert xs.shape == mask.shape
    xs = paddle.where(mask, trues, xs)
    return xs

def masked_fill_(xs:paddle.Tensor, mask:paddle.Tensor, value:float):
    print('x', xs)
    trues = paddle.ones_like(xs) * value
    assert xs.shape == mask.shape
    ret = paddle.where(mask, trues, xs)
    print('2', xs)
    paddle.assign(ret, output=xs)
    print('3', xs)

paddle.Tensor.masked_fill = masked_fill
paddle.Tensor.masked_fill_ = masked_fill_

def mask_finished_scores_pd(score: paddle.Tensor,
                         flag: paddle.Tensor) -> paddle.Tensor:
    """
    If a sequence is finished, we only allow one alive branch. This function
    aims to give one branch a zero score and the rest -inf score.
    Args:
        score (torch.Tensor): A real value array with shape
            (batch_size * beam_size, beam_size).
        flag (torch.Tensor): A bool array with shape
            (batch_size * beam_size, 1).
    Returns:
        torch.Tensor: (batch_size * beam_size, beam_size).
    """
    beam_size = score.shape[-1]
    zero_mask = paddle.zeros_like(flag, dtype=paddle.bool)
    if beam_size > 1:
        unfinished = paddle.concat((zero_mask, flag.tile([1, beam_size - 1])),
                               axis=1)
        finished = paddle.concat((flag, zero_mask.tile([1, beam_size - 1])),
                             axis=1)
    else:
        unfinished = zero_mask
        finished = flag
    print(unfinished)
    print(finished)
    
    #score.masked_fill_(unfinished, -float('inf'))
    #score.masked_fill_(finished, 0)
#     infs = paddle.ones_like(score) * -float('inf')
#     score = paddle.where(unfinished, infs, score)
#     score = paddle.where(finished, paddle.zeros_like(score), score)

#     score = score.masked_fill(unfinished, -float('inf'))
#     score = score.masked_fill(finished, 0)
    score.masked_fill_(unfinished, -float('inf'))
    score.masked_fill_(finished, 0)
    return score

r  = mask_finished_scores_pd(score, flag)
print(r)

Tensor(shape=[2, 3], dtype=bool, place=CUDAPlace(0), stop_gradient=True,
       [[False, True , True ],
        [False, False, False]])
Tensor(shape=[2, 3], dtype=bool, place=CUDAPlace(0), stop_gradient=True,
       [[True , False, False],
        [False, False, False]])
x Tensor(shape=[2, 3], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [[ 2.05994511,  1.87704289,  0.01988174],
        [-0.40165186,  0.77547729, -0.64469045]])
2 Tensor(shape=[2, 3], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [[ 2.05994511,  1.87704289,  0.01988174],
        [-0.40165186,  0.77547729, -0.64469045]])
3 Tensor(shape=[2, 3], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [[ 2.05994511, -inf.      , -inf.      ],
        [-0.40165186,  0.77547729, -0.64469045]])
x Tensor(shape=[2, 3], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [[ 2.05994511, -inf.      , -inf.      ],
        [-0.40165186,  0.77547729, -0.64469045]])
2 Tensor(shape=[2, 3

In [57]:
score.value

<bound method PyCapsule.value of Tensor(shape=[2, 3], dtype=float32, place=CUDAPlace(0), stop_gradient=True,
       [[ 0.        , -inf.      , -inf.      ],
        [-0.40165186,  0.77547729, -0.64469045]])>

In [71]:
from typing import Union, Any

In [72]:
def repeat(xs : paddle.Tensor, *size: Any):
    print(size)
    return paddle.tile(xs, size)
paddle.Tensor.repeat = repeat

In [73]:
flag = paddle.ones((2, 1), dtype='bool')
flag[1] = False
print(flag.repeat(1, 2))

(1, 2)
Tensor(shape=[2, 2], dtype=bool, place=CUDAPlace(0), stop_gradient=True,
       [[True , True ],
        [False, False]])


In [84]:
flag = paddle.ones((2, 1), dtype='bool')
flag[1] = False
print(flag.repeat(paddle.to_tensor(1), 2))

(Tensor(shape=[1], dtype=int64, place=CUDAPlace(0), stop_gradient=True,
       [1]), 2)
Tensor(shape=[2, 2], dtype=bool, place=CUDAPlace(0), stop_gradient=True,
       [[True , True ],
        [False, False]])


In [85]:
def size(xs, *args: int):
    nargs = len(args)
    s = paddle.shape(xs)
    assert(nargs <= 1)
    if nargs == 1:
        return s[args[0]]
    else:
        return s
paddle.Tensor.size = size

In [86]:
flag.size()

Tensor(shape=[2], dtype=int32, place=CPUPlace, stop_gradient=True,
       [2, 1])

In [87]:
flag.size(1)

Tensor(shape=[1], dtype=int32, place=CPUPlace, stop_gradient=True,
       [1])

In [88]:
flag.size(0)

Tensor(shape=[1], dtype=int32, place=CPUPlace, stop_gradient=True,
       [2])